In [1]:
import ROOT
from fitter import FitHandler

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x8893e40


In [2]:
# TODO: make it possible to fit multiple runs simultaneously
run = {
    "lumi": 5000,
    # "lumi": 25000,
    "e_pol": 0.,
    "p_pol": 0.,
}

# https://arxiv.org/pdf/1506.07830
# However, https://arxiv.org/pdf/2503.19983 cites the above but assigns 0.45 to both mixed pols
# LCVision scenario uses 3 ab_inv instead of 2
ilc_250_h20_lumi = 2000
ilc_250_h20 = [
    {
        "lumi": ilc_250_h20_lumi * 0.675,
        "e_pol": -0.8,
        "p_pol": 0.3,
    },
    {
        "lumi": ilc_250_h20_lumi * 0.225,
        "e_pol": 0.8,
        "p_pol": -0.3,
    },
    {
        "lumi": ilc_250_h20_lumi * 0.05,
        "e_pol": -0.8,
        "p_pol": -0.3,
    },
    {
        "lumi": ilc_250_h20_lumi * 0.05,
        "e_pol": 0.8,
        "p_pol": 0.3,
    },
]

# run = ilc_250_h20[0]

In [3]:
conf = {
    "parameters": {
        "g1z": 0.0,
        "ka": 0.0,
        "la": 0.0,
    },
    "obs_names": [
        "O_g1z_pos_1em05",
        "O_ka_pos_1em05",
        "O_la_pos_1em05",
    ],
    "signal_cat": "4f_sw_sl_signal",
    "signal_processes": [
        "4f_sw_sl_eLpL_signal",
        "4f_sw_sl_eLpR_signal",
        "4f_sw_sl_eRpL_signal",
        "4f_sw_sl_eRpR_signal",
    ],
    "backgrounds": [
        "4f_sl_bkg",
        "4f_not_sl",
        "2f",
        "3f",
        "5f",
        "6f",
        "higgs",
    ],
    "n_bins": 65,
}
input_path = "data/histograms/full/raw_histograms.root"

In [4]:
fit_handler = FitHandler(input_path, conf)

In [5]:
test_fun,_ = fit_handler.make_expected_fun(conf["obs_names"][0], run)

start constructor
finished constructor


In [6]:
pars = [1., 0., 0., 0., 0., 0., 1.]
print(test_fun(pars))
pars = [1., -0.1, 0.01, 0., 0., 0., 1.]
print(test_fun(pars))

-328312.7900271484
-364851.6328907656


In [7]:
ws = fit_handler.build_model(run)
# coupling_pars = fit_handler.coupling_pars
coupling_pars = [ws.var(name) for name in conf["parameters"]]
obs_pars = [ws.var(name) for name in conf["obs_names"]]

hello
start constructor
finished constructor
start constructor
finished constructor
start constructor
finished constructor
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooMultiVarGaussian::multi_gauss
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::O_g1z_pos_1em05
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::O_ka_pos_1em05
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::O_la_pos_1em05
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooFunctorBinding::expectation_O_g1z_pos_1em05
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::lumi
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::e_pol
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::p_pol
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::g1z
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::ka
[

In [8]:
ws.Print("t")


RooWorkspace(ws) ws contents

variables
---------
(O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05,e_pol,g1z,ka,la,lumi,mu_signal,p_pol)

p.d.f.s
-------
RooMultiVarGaussian::multi_gauss[ x=(O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05) mu=(expectation_O_g1z_pos_1em05,expectation_O_ka_pos_1em05,expectation_O_la_pos_1em05) ] = 1
  RooFunctorBinding::expectation_O_g1z_pos_1em05[ function=0x21f20440 vars=(lumi,e_pol,p_pol,g1z,ka,la,mu_signal) ] = -328313
  RooFunctorBinding::expectation_O_ka_pos_1em05[ function=0x228589d0 vars=(lumi,e_pol,p_pol,g1z,ka,la,mu_signal) ] = -816884
  RooFunctorBinding::expectation_O_la_pos_1em05[ function=0x2285d650 vars=(lumi,e_pol,p_pol,g1z,ka,la,mu_signal) ] = -404445



In [9]:
model = ws.pdf("multi_gauss")

In [10]:
ds = ROOT.RooStats.AsymptoticCalculator.GenerateAsimovData(model, obs_pars)
ds.Print("v")

DataStore CountingAsimovData0 (CountingAsimovData0)
  Contains 1 entries
  Observables: 
    1)  O_g1z_pos_1em05 = -328313  L(-334268 - -322357)  "O_g1z_pos_1em05"
    2)   O_ka_pos_1em05 = -816884  L(-822088 - -811680)  "O_ka_pos_1em05"
    3)   O_la_pos_1em05 = -404445  L(-412478 - -396412)  "O_la_pos_1em05"


In [11]:
fit_res = model.fitTo(ds, Save=True)

[#1] INFO:Fitting -- RooAbsPdf::fitTo(multi_gauss_over_multi_gauss_Int[O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 6.78829 ms
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_multi_gauss_over_multi_gauss_Int[O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 1500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 25.5693367216680087
Edm   = 4.35788018137643373e-26
Nfcn  = 40
g1z	  = 0	 +/-  0.000880172	(limited)
ka	  = 0	 +/-  0.0011991	(limited)
la	  = 0	 +/-  0.000861372	(limited)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setO

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 254.761 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       25.56933672 Edm =   3.618396539e-26 NCalls =     13
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 25.56933672
  Edm           : 3.618396539e-26
  Internal parameters:	[                0                0                0]	
  Internal gradient  :	[  -7.93969837e-11  9.704075785e-11  1.058626449e-10]	
  Internal covariance matrix:
[[  2.7922202e-06              0              0]
 [              0  1.0083714e-05              0]
 [              0              0  2.8711442e-06]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       25.56933672 Edm =   3.618396539e-26 NCalls =     13
Warning in <Minuit2>: VariableMetricBuilder No improvemen

In [12]:
fit_res.covarianceMatrix().Print()


3x3 matrix is as follows

     |      0    |      1    |      2    |
--------------------------------------------
   0 |  7.747e-07  -2.769e-07   5.286e-07 
   1 | -2.769e-07   1.438e-06  -1.601e-08 
   2 |  5.286e-07  -1.601e-08    7.42e-07 



In [13]:
nll = model.createNLL(ds, EvalBackend="cpu")

[#1] INFO:Fitting -- RooAbsPdf::fitTo(multi_gauss_over_multi_gauss_Int[O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- Creation of NLL object took 215.374 μs


In [14]:
nll_minimizer = ROOT.RooMinimizer(nll)

[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_multi_gauss_over_multi_gauss_Int[O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level


In [15]:
%time
nll_minimizer.migrad()

CPU times: user 11 μs, sys: 0 ns, total: 11 μs
Wall time: 20.3 μs


0

Minuit2Minimizer: Minimize with max-calls 1500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 25.5693367216680087
Edm   = 4.01127747495378121e-25
Nfcn  = 40
g1z	  = 0	 +/-  0.000880172	(limited)
ka	  = 0	 +/-  0.0011991	(limited)
la	  = 0	 +/-  0.000861372	(limited)


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 485.827 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       25.56933672 Edm =   2.538401714e-25 NCalls =     13
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 25.56933672
  Edm           : 2.538401714e-25
  Internal parameters:	[                0                0                0]	
  Internal gradient  :	[                0  -2.24380401e-10 -4.205016199e-10]	
  Internal covariance matrix:
[[  2.7922202e-06              0              0]
 [              0  1.0083714e-05              0]
 [              0              0  2.8711442e-06]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       25.56933672 Edm =   2.538401714e-25 NCalls =     13
Warning in <Minuit2>: VariableMetricBuilder No improvemen

In [16]:
%time
# nll_minimizer.hesse()
# nll_minimizer.minos(coupling_pars)

CPU times: user 8 μs, sys: 2 μs, total: 10 μs
Wall time: 15.5 μs


In [17]:
# nll.Print("t")

In [18]:
pll0 = nll.createProfile({coupling_pars[0]})
pll1 = nll.createProfile({coupling_pars[1]})
pll2 = nll.createProfile({coupling_pars[2]})

In [19]:
frame0 = coupling_pars[0].frame(Range=(-0.004, 0.004))
frame1 = coupling_pars[1].frame(Range=(-0.004, 0.004))
frame2 = coupling_pars[2].frame(Range=(-0.004, 0.004))
nll.plotOn(frame0, ShiftToZero=True)
nll.plotOn(frame1, ShiftToZero=True)
nll.plotOn(frame2, ShiftToZero=True)

In [20]:
pll0.plotOn(frame0, LineColor="r")
pll1.plotOn(frame1, LineColor="r")
pll2.plotOn(frame2, LineColor="r")

[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_multi_gauss_over_multi_gauss_Int[O_g1z_pos_1em05,O_ka_pos_1em05,O_la_pos_1em05]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) minimum found at (g1z=0)
..........................................................................................................................................................................................................
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[ka]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_multi_gauss_over_multi_gauss_Int[O_g1z_

In [21]:
c0 = ROOT.TCanvas()
frame0.SetMinimum(0)
frame0.SetMaximum(4)
frame0.Draw()
c0.Draw()
# c0.SaveAs("plots/fit/ll_pll.pdf(")
# c0.SaveAs("plots/fit/ll_pll_g1z.pdf")

c1 = ROOT.TCanvas()
frame1.SetMinimum(0)
frame1.SetMaximum(4)
frame1.Draw()
c1.Draw()
# c1.SaveAs("plots/fit/ll_pll.pdf")
# c1.SaveAs("plots/fit/ll_pll_ka.pdf")

c2 = ROOT.TCanvas()
frame2.SetMinimum(0)
frame2.SetMaximum(4)
frame2.Draw()
c2.Draw()
# c2.SaveAs("plots/fit/ll_pll.pdf)")
# c2.SaveAs("plots/fit/ll_pll_la.pdf")